In [1]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


In [10]:
model = pyo.ConcreteModel()

model.produtos = pyo.Set(initialize =['Produto A','Produto B'])
model.recursos = pyo.Set(initialize =['Q','R','S'])

model.x= pyo.Var(model.produtos, domain=NonNegativeReals)

necessidade_recursos={
    'Produto A': {'Q': 2, 'R': 1,'S':3},
    'Produto B': {'Q': 1, 'R': 2,'S':3}
}

quantidade_recursos={
    'Q': 2,
    'R': 2,
    'S': 4
}

lucro={
    'Produto A': 3,
    'Produto B': 2
}

#objetivo
def objetivo(model):
    return sum(lucro[p] * model.x[p] for p in model.produtos)   
model.objetivo = pyo.Objective(rule=objetivo, sense=maximize)

#restrições
def restricao_recursos(model, r):
    return sum(necessidade_recursos[p][r] * model.x[p] for p in model.produtos) <= quantidade_recursos[r]   
model.restricao_recursos = pyo.Constraint(model.recursos, rule=restricao_recursos)

#resolver o modelo
opt = SolverFactory('cplex')  
results = opt.solve(model)

In [11]:
model.pprint()

2 Set Declarations
    produtos : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Produto A', 'Produto B'}
    recursos : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Q', 'R', 'S'}

1 Var Declarations
    x : Size=2, Index=produtos
        Key       : Lower : Value              : Upper : Fixed : Stale : Domain
        Produto A :     0 : 0.6666666666666667 :  None : False : False : NonNegativeReals
        Produto B :     0 : 0.6666666666666665 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objetivo : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 3*x[Produto A] + 2*x[Produto B]

1 Constraint Declarations
    restricao_recursos : Size=3, Index=recursos, Active=True
        Key : Lower : Body                            : Upper : Active
    

In [12]:
for p in model.produtos:
    print(f"{p}: {model.x[p].value}")

print(f"Lucro total: {pyo.value(model.objetivo)}")

Produto A: 0.6666666666666667
Produto B: 0.6666666666666665
Lucro total: 3.333333333333333
